In [1]:
!git config --global user.name "Paris203"
!git config --global user.email "bolongott@gmail.com"

In [2]:
!pwd

/kaggle/working


In [3]:
!git clone https://github.com/Paris203/Karlos.git

Cloning into 'Karlos'...
remote: Enumerating objects: 12686, done.
remote: Counting objects: 100% (651/651), done.
remote: Compressing objects: 100% (381/381), done.
remote: Total 12686 (delta 338), reused 526 (delta 270), pack-reused 12035 (from 1)
Receiving objects: 100% (12686/12686), 1.15 GiB | 48.33 MiB/s, done.
Resolving deltas: 100% (343/343), done.
Updating files: 100% (11877/11877), done.


In [4]:
%cd /kaggle/working/Karlos

/kaggle/working/Karlos


In [5]:
ls -l

total 36
drwxr-xr-x 4 root root 4096 Sep  9 05:52 checkpoint/
-rw-r--r-- 1 root root 2555 Sep  9 05:52 config.py
drwxr-xr-x 5 root root 4096 Sep  9 05:52 datasets_m/
drwxr-xr-x 2 root root 4096 Sep  9 05:52 envconfig/
drwxr-xr-x 3 root root 4096 Sep  9 05:52 models/
drwxr-xr-x 2 root root 4096 Sep  9 05:52 networks/
-rw-r--r-- 1 root root 1808 Sep  9 05:52 test.py
-rw-r--r-- 1 root root 2270 Sep  9 05:52 train.py
drwxr-xr-x 2 root root 4096 Sep  9 05:52 utils/


In [6]:
!python train.py

Resized activation map shape: torch.Size([6, 48, 48])
Loading CUB trainset
/opt/conda/lib/python3.10/site-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
Loading CUB testset
/kaggle/working/Karlos/networks/resnet.py:248: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default valu

In [7]:
# ma
def compute_cam(feature_maps, classifier_weights):
    """
    Compute the Class Activation Map (CAM).
    
    Args:
        feature_maps (torch.Tensor): Feature maps from the last convolutional layer (shape: [batch_size, num_channels, height, width]).
        classifier_weights (torch.Tensor): Weights of the classifier corresponding to the class (shape: [num_channels]).
    
    Returns:
        torch.Tensor: The CAM for the given input (shape: [batch_size, height, width]).
    """
    # Compute the weighted sum of the feature maps
    weighted_sum = torch.einsum('ijkl,j->ikl', feature_maps, classifier_weights)
    
    # Apply the ReLU function to match the dimensions and normalize
    cam = F.relu(weighted_sum)
    
    return cam


def compute_spm(cam):
    """
    Compute the Semantic Percent Map (SPM).
    
    Args:
        cam (torch.Tensor): The Class Activation Map (CAM) (shape: [batch_size, height, width]).
    
    Returns:
        torch.Tensor: The SPM for the given input (shape: [batch_size, height, width]).
    """
    # Normalize CAM by its sum to compute SPM
    spm = cam / cam.sum(dim=(1, 2), keepdim=True)
    
    return spm

def compute_label_weights(spm, mask_a, mask_b):
    """
    Compute the label weights ρa and ρb.
    
    Args:
        spm (torch.Tensor): The SPM for the given input (shape: [batch_size, height, width]).
        mask_a (torch.Tensor): Binary mask for region of interest φa (shape: [batch_size, height, width]).
        mask_b (torch.Tensor): Binary mask for region of interest φb (shape: [batch_size, height, width]).
    
    Returns:
        Tuple[torch.Tensor, torch.Tensor]: The label weights ρa and ρb.
    """
    # Compute ρa
    rho_a = 1 - torch.sum(mask_a * spm, dim=(1, 2))
    
    # Compute ρb
    rho_b = torch.sum(mask_b * spm, dim=(1, 2))
    
    return rho_a, rho_b
